## Импорт необходимого функционала

In [1]:
import os
import pickle
import numpy as np
import pandas as pd

## Загрузка данных и сохранённых весов классификаторов

In [2]:
test = pd.read_csv("data/mars-private_test-class.csv")

list_models = os.listdir("models/")
all_models = {1: [], 2: [], 3: []}

for model_path in list_models:
    with open(f"models/{model_path}", "rb") as file:
        model = pickle.load(file)
    prefix = int(model_path[0])
    all_models[prefix].append(model)
    
del list_models

## Предсказание на тесте

In [3]:
np.random.seed(5432)

test["Тип марсианина"] = -1
for i in range(1, 4):
    estimators = all_models[i]
    test_trial = test[test["№ испытания"].values == i].drop(["Тип марсианина", "№ испытания",], axis=1)
    idx = test_trial.index
    trial_preds = np.empty((test_trial.shape[0], len(estimators)))
    for i, estimator in enumerate(estimators):
        trial_preds[:, i] = estimator.predict(test_trial)
    test.loc[idx, "Тип марсианина"] = pd.DataFrame(trial_preds).mode(axis=1).values.ravel()
    
assert not(-1 in test["Тип марсианина"].unique())

In [4]:
test["Тип марсианина"].value_counts() / test.shape[0]

1    0.900196
0    0.099804
Name: Тип марсианина, dtype: float64

In [5]:
test.to_csv("solution_class.csv", index=False)

---